In [13]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
import os
from sklearn.metrics import log_loss

In [14]:
d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


# 전처리

### family_type
- 'Married', 'Civil marriage'
- 'Separated',  'Widow'
- 'Single / not married'

In [15]:
# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

In [16]:
def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

In [17]:
# 안 덮어쓰기
for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

In [18]:
# income_total을 로그변환 한 새로운 feature log_income
# 기존 칼럼 삭제
train['log_income'] = np.log(train['income_total'])
train = train.drop('income_total',1)
test['log_income'] = np.log(test['income_total'])
test = test.drop('income_total',1)

In [19]:
# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

In [20]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [21]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [22]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

# 모델링, 학습

In [23]:
len(train.columns)

58

In [25]:
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
folds=[]
losses=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    losses.append(log_loss(y_valid, lgb.predict_proba(X_valid)))
    print(f'================================================================================\n\n')
print(sum(losses)/5)

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.658987	valid_1's multi_logloss: 0.732107
[200]	training's multi_logloss: 0.576659	valid_1's multi_logloss: 0.714321
[300]	training's multi_logloss: 0.518744	valid_1's multi_logloss: 0.707771
[400]	training's multi_logloss: 0.472305	valid_1's multi_logloss: 0.705282
Early stopping, best iteration is:
[382]	training's multi_logloss: 0.480272	valid_1's multi_logloss: 0.70432


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.659654	valid_1's multi_logloss: 0.733596
[200]	training's multi_logloss: 0.579481	valid_1's multi_logloss: 0.711939
[300]	training's multi_logloss: 0.522213	valid_1's multi_logloss: 0.703459
[400]	training's multi_logloss: 0.474595	valid_1's multi_logloss: 0.70085
Earl

###
- 5fold: 0.7327810945192511
- 10fold: 0.7294648153551373  0.04차이남
- 20fold: 

In [30]:
ss.iloc[:,1:]=0
for fold in range(5):
    ss.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [31]:
ss.to_csv('5_marriage_type.csv', index=False)